In [6]:
from PIL import Image
from PIL.ExifTags import TAGS, GPSTAGS
import os
import re
import csv
from shapely.geometry import Point, shape
import fiona
import numpy as np

input_photo_folder = r"F:\LM2403-66 Dahrouge EGNC\Pix4DMatic processing\Eleonore 1\exports"


# Suppress DecompressionBombWarning
Image.MAX_IMAGE_PIXELS = None

def get_exif_data(image_path):
    image = Image.open(image_path)
    exif_data = image._getexif()
    if not exif_data:
        return None

    exif = {}
    for tag, value in exif_data.items():
        decoded = TAGS.get(tag, tag)
        exif[decoded] = value
    return exif

def get_gps_data(image_path):
    exif_data = get_exif_data(image_path)
    gps_info = {}
    for key in exif_data['GPSInfo'].keys():
        decode = GPSTAGS.get(key, key)
        gps_info[decode] = exif_data['GPSInfo'][key]
    latitude, longitude = extract_lat_lon(gps_info)
    return latitude, longitude

def convert_to_degrees(value):
    d, m, s = value
    return d + (m / 60.0) + (s / 3600.0)

def extract_lat_lon(gps_data):
    lat = None
    lon = None
    if gps_data:
        gps_latitude = gps_data.get('GPSLatitude')
        gps_latitude_ref = gps_data.get('GPSLatitudeRef')
        gps_longitude = gps_data.get('GPSLongitude')
        gps_longitude_ref = gps_data.get('GPSLongitudeRef')

        if gps_latitude and gps_latitude_ref and gps_longitude and gps_longitude_ref:
            lat = convert_to_degrees(gps_latitude)
            if gps_latitude_ref != 'N':
                lat = -lat

            lon = convert_to_degrees(gps_longitude)
            if gps_longitude_ref != 'E':
                lon = -lon

    return lat, lon

def get_list_of_paths_os_walk_folder(folder_path, ext):
    file_paths = []
    for root, dirs, files in os.walk(folder_path):
        for filename in files:
            if os.path.splitext(filename)[1].lower() == ext.lower():
                file_path = os.path.join(root, filename)
                file_paths.append(file_path)

    def natural_sort_key(s):
        # This function will create a sort key that handles numbers properly
        return [int(text) if text.isdigit() else text.lower() for text in re.split('(\d+)', s)]

    file_paths.sort(key=natural_sort_key)
    return file_paths

def get_name_of_non_existing_output_file(base_filepath, additional_suffix='', new_extention=''):
    # Function to create a unique file path by adding a version number
    base, ext = os.path.splitext(base_filepath)
    if new_extention:
        ext = new_extention
    new_out_file_path = f"{base}{additional_suffix}{ext}"

    if not os.path.exists(new_out_file_path):
        return new_out_file_path

    version = 2
    while os.path.exists(f"{base}{additional_suffix}_v{version}{ext}"):
        version += 1
    return f"{base}{additional_suffix}_v{version}{ext}"

def save_to_csv(output_csv_file, *args, column_names=None):
    # Check if all lists have the same length
    if not all(len(lst) == len(args[0]) for lst in args):
        print("Error: All input lists must be of the same length.")
        return

    # Set default column names if none are provided
    if column_names is None:
        column_names = ['Column' + str(i+1) for i in range(len(args))]
    elif len(column_names) != len(args):
        print("Error: Number of column names must match number of input lists.")
        return

    # Open the CSV file for writing
    with open(output_csv_file, mode='w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)

        # Write the header row based on the provided or default column names
        csv_writer.writerow(column_names)

        # Write the data rows
        for row in zip(*args):
            csv_writer.writerow(row)

    print(f"Data has been saved to {output_csv_file}")


def get_photos_in_polygons(tile_polygon_paths, lat_list, lon_list, photo_paths):
    # Initialize the boolean mask list with False values
    boolean_mask_dict = {}

    for tile_polygon_path in tile_polygon_paths:
        boolean_mask = [False] * len(photo_paths)
        # Open the shapefile and iterate over its features
        with fiona.open(tile_polygon_path) as shapefile:
            print('new shp')
            for feature in shapefile:
                polygon = shape(feature['geometry'])

                # Check each photo's coordinates
                for i, (lat, lon, photo_path) in enumerate(zip(lat_list, lon_list, photo_paths)):
                    point = Point(lon, lat)
                    if polygon.contains(point):
                        boolean_mask[i] = True
                        print(f'Photo {photo_path} is in a polygon.')
                    else:
                        print(f'Photo {photo_path} is not in a polygon.')
        boolean_mask_dict[tile_polygon_path] = boolean_mask
    return boolean_mask_dict


photo_paths = get_list_of_paths_os_walk_folder(input_photo_folder, '.tiff')

lat_list, lon_list = [], []
for path in photo_paths:
    latitude, longitude = get_gps_data(path)
    #print(f"Latitude: {latitude}, Longitude: {longitude}")
    lat_list.append(latitude)
    lon_list.append(longitude)

if not len(lat_list) == len(lon_list) == len(photo_paths) > 0:
    raise ValueError

boolean_mask_dict = get_photos_in_polygons(tile_polygon_paths, lat_list, lon_list, photo_paths)
for poly_path, mask in boolean_mask_dict.items():
    print(poly_path, mask)
    name = os.path.join(os.path.dirname(os.path.dirname(poly_path)), os.path.basename(poly_path))
    output_csv_file = get_name_of_non_existing_output_file(name,
                                                           additional_suffix='_photo_locations',
                                                           new_extention='.csv')
    poly_paths =  np.array(photo_paths)[mask].tolist()
    poly_lons = np.array(lon_list)[mask].tolist()
    poly_lats =  np.array(lat_list)[mask].tolist()
    save_to_csv(output_csv_file, poly_paths, poly_lons, poly_lats, column_names=['Name', 'Longitude', 'Latitude'])

ModuleNotFoundError: No module named 'shapely'

In [40]:
from PIL import Image
from PIL.ExifTags import TAGS, GPSTAGS
import os
import re
import csv
from shapely.geometry import Point, shape
import fiona
import numpy as np

input_photo_folder = r"F:\LM2403-66 Dahrouge EGNC\Photos within flightlines\Eleonore flight 2"


# Suppress DecompressionBombWarning
Image.MAX_IMAGE_PIXELS = None

def get_exif_data(image_path):
    image = Image.open(image_path)
    exif_data = image._getexif()
    if not exif_data:
        return None

    exif = {}
    for tag, value in exif_data.items():
        decoded = TAGS.get(tag, tag)
        exif[decoded] = value
    return exif
    

In [42]:
def get_gps_data(image_path):
    exif_data = get_exif_data(image_path)
    gps_info = {}
    for key in exif_data['GPSInfo'].keys():
        decode = GPSTAGS.get(key, key)
        gps_info[decode] = exif_data['GPSInfo'][key]
    latitude, longitude = extract_lat_lon(gps_info)
    return latitude, longitude

def convert_to_degrees(value):
    d, m, s = value
    return d + (m / 60.0) + (s / 3600.0)

def extract_lat_lon(gps_data):
    lat = None
    lon = None
    if gps_data:
        gps_latitude = gps_data.get('GPSLatitude')
        gps_latitude_ref = gps_data.get('GPSLatitudeRef')
        gps_longitude = gps_data.get('GPSLongitude')
        gps_longitude_ref = gps_data.get('GPSLongitudeRef')

        if gps_latitude and gps_latitude_ref and gps_longitude and gps_longitude_ref:
            lat = convert_to_degrees(gps_latitude)
            if gps_latitude_ref != 'N':
                lat = -lat

            lon = convert_to_degrees(gps_longitude)
            if gps_longitude_ref != 'E':
                lon = -lon

    return lat, lon

In [44]:
def get_list_of_paths_os_walk_folder(folder_path, ext):
    file_paths = []
    for root, dirs, files in os.walk(folder_path):
        for filename in files:
            if os.path.splitext(filename)[1].lower() == ext.lower():
                file_path = os.path.join(root, filename)
                file_paths.append(file_path)

    def natural_sort_key(s):
        # This function will create a sort key that handles numbers properly
        return [int(text) if text.isdigit() else text.lower() for text in re.split('(\d+)', s)]

    file_paths.sort(key=natural_sort_key)
    return file_paths

In [46]:
def get_name_of_non_existing_output_file(base_filepath, additional_suffix='', new_extention=''):
    # Function to create a unique file path by adding a version number
    base, ext = os.path.splitext(base_filepath)
    if new_extention:
        ext = new_extention
    new_out_file_path = f"{base}{additional_suffix}{ext}"

    if not os.path.exists(new_out_file_path):
        return new_out_file_path

    version = 2
    while os.path.exists(f"{base}{additional_suffix}_v{version}{ext}"):
        version += 1
    return f"{base}{additional_suffix}_v{version}{ext}"


In [48]:

def save_to_csv(output_csv_file, *args, column_names=None):
    # Check if all lists have the same length
    if not all(len(lst) == len(args[0]) for lst in args):
        print("Error: All input lists must be of the same length.")
        return
# Set default column names if none are provided
    if column_names is None:
        column_names = ['Column' + str(i+1) for i in range(len(args))]
    elif len(column_names) != len(args):
        print("Error: Number of column names must match number of input lists.")
        return

    # Open the CSV file for writing
    with open(output_csv_file, mode='w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)

        # Write the header row based on the provided or default column names
        csv_writer.writerow(column_names)

        # Write the data rows
        for row in zip(*args):
            csv_writer.writerow(row)

    print(f"Data has been saved to {output_csv_file}")


In [50]:
def get_photos_in_polygons(tile_polygon_paths, lat_list, lon_list, photo_paths):
    # Initialize the boolean mask list with False values
    boolean_mask_dict = {}

    for tile_polygon_path in tile_polygon_paths:
        boolean_mask = [False] * len(photo_paths)
        # Open the shapefile and iterate over its features
        with fiona.open(tile_polygon_path) as shapefile:
            print('new shp')
            for feature in shapefile:
                polygon = shape(feature['geometry'])

                # Check each photo's coordinates
                for i, (lat, lon, photo_path) in enumerate(zip(lat_list, lon_list, photo_paths)):
                    point = Point(lon, lat)
                    if polygon.contains(point):
                        boolean_mask[i] = True
                        print(f'Photo {photo_path} is in a polygon.')
                    else:
                        print(f'Photo {photo_path} is not in a polygon.')
        boolean_mask_dict[tile_polygon_path] = boolean_mask
    return boolean_mask_dict

In [52]:
photo_paths = get_list_of_paths_os_walk_folder(input_photo_folder, '.tiff')

lat_list, lon_list = [], []
for path in photo_paths:
    latitude, longitude = get_gps_data(path)
    #print(f"Latitude: {latitude}, Longitude: {longitude}")
    lat_list.append(latitude)
    lon_list.append(longitude)

if not len(lat_list) == len(lon_list) == len(photo_paths) > 0:
    raise ValueError

boolean_mask_dict = get_photos_in_polygons(tile_polygon_paths, lat_list, lon_list, photo_paths)
for poly_path, mask in boolean_mask_dict.items():
    print(poly_path, mask)
    name = os.path.join(os.path.dirname(os.path.dirname(poly_path)), os.path.basename(poly_path))
    output_csv_file = get_name_of_non_existing_output_file(name,
                                                           additional_suffix='_photo_locations',
                                                           new_extention='.csv')
    poly_paths =  np.array(photo_paths)[mask].tolist()
    poly_lons = np.array(lon_list)[mask].tolist()
    poly_lats =  np.array(lat_list)[mask].tolist()
    save_to_csv(output_csv_file, poly_paths, poly_lons, poly_lats, column_names=['Name', 'Longitude', 'Latitude'])

ValueError: 